## Code for foursquare format to Unitrip format (temp in notebook)

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import osmnx as ox
import pandas as pd
import geopandas as gpd

%matplotlib inline
sns.set(context='notebook', font='Lucida Sans Unicode', style='white', palette='plasma')

Import foursquare data (raw POIs)

In [4]:
pois = pd.read_csv('../data/other_format/foursquare/raw_POIs.txt', sep='\t', names=['venue_id', 'lat', 'lon', 'category', 'country']) 
pois

,venue_id,lat,lon,category,country
0,3fd66200f964a52000e61ee3,40.729209,-73.998753,Post Office,US
1,3fd66200f964a52000e71ee3,40.733596,-74.003139,Jazz Club,US
2,3fd66200f964a52000e81ee3,40.758102,-73.975734,Gym,US
3,3fd66200f964a52000ea1ee3,40.732456,-74.003755,Indian Restaurant,US
4,3fd66200f964a52000ec1ee3,42.345907,-71.087001,Indian Restaurant,US
...,...,...,...,...,...
11180155,52e92ec0498e1a09484475c9,55.852631,37.447034,Residential Building (Apartment / Condo),RU
11180156,52e92f11498e8e25136e9fe2,-4.329570,-40.711533,Dentist's Office,BR
11180157,52e92f4511d2a77832a49eec,52.505471,13.368507,Financial or Legal Service,DE
11180158,52e92fa5498ef926067570ff,41.025367,28.554854,City,TR


filter the POIs data with the municipalities of interest

In [5]:
municipalities = '''Cerrillos
La Reina
Pudahuel
Cerro Navia
Las Condes
Quilicura
Conchalí
Lo Barnechea
Quinta Normal
El Bosque
Lo Espejo
Recoleta
Estación Central
Lo Prado
Renca
Huechuraba
Macul
San Miguel (Chile)
Independencia (Chile)
Maipú
San Joaquín (Chile)
La Cisterna
Ñuñoa
San Ramón (Chile)
La Florida
Pedro Aguirre Cerda
Santiago de Chile
La Pintana
Peñalolén
Vitacura
La Granja (Chile)
Providencia
Peñaflor (Chile)
San Bernardo (Chile)
Padre Hurtado
Puente Alto
'''.strip().split('\n')

amb = ox.geocoder.geocode_to_gdf(municipalities)

In [15]:
bbox = amb.total_bounds
pois_in_bbox = pois[pois.lon.between(bbox[0], bbox[2]) & pois.lat.between(bbox[1], bbox[3])]
pois_in_bbox

,venue_id,lat,lon,category,country
109597,4af8ffeaf964a520cd1022e3,-33.446160,-70.660458,Other Great Outdoors,CL
222118,4b4463faf964a52078f425e3,-33.373803,-70.517930,Sushi Restaurant,CL
222614,4b44abe7f964a5201ef925e3,-33.412225,-70.577652,Sushi Restaurant,CL
222624,4b44ad89f964a52040f925e3,-33.404482,-70.572755,Office,CL
222705,4b44b566f964a5200efa25e3,-33.430976,-70.622628,Sandwich Place,CL
...,...,...,...,...,...
11179377,52e8241811d24e40d89978d0,-33.476055,-70.603833,Home (private),CL
11179605,52e8d7ad498e459461a0b33b,-33.423755,-70.727859,Bus Line,CL
11179962,52e90de1498ed1ed01b47aaf,-33.448690,-70.843062,Bus Line,CL
11179993,52e912e9498ed5442c097af5,-33.392693,-70.617067,Campaign Office,CL


In [16]:
pois_gdf = gpd.GeoDataFrame(pois_in_bbox[['venue_id', 'category']],
                            geometry=gpd.points_from_xy(pois_in_bbox.lon, pois_in_bbox.lat),
                            crs='EPSG:4326')
pois_gdf

,venue_id,category,geometry
109597,4af8ffeaf964a520cd1022e3,Other Great Outdoors,POINT (-70.66046 -33.44616)
222118,4b4463faf964a52078f425e3,Sushi Restaurant,POINT (-70.51793 -33.3738)
222614,4b44abe7f964a5201ef925e3,Sushi Restaurant,POINT (-70.57765 -33.41222)
222624,4b44ad89f964a52040f925e3,Office,POINT (-70.57276 -33.40448)
222705,4b44b566f964a5200efa25e3,Sandwich Place,POINT (-70.62263 -33.43098)
...,...,...,...
11179377,52e8241811d24e40d89978d0,Home (private),POINT (-70.60383 -33.47606)
11179605,52e8d7ad498e459461a0b33b,Bus Line,POINT (-70.72786 -33.42376)
11179962,52e90de1498ed1ed01b47aaf,Bus Line,POINT (-70.84306 -33.44869)
11179993,52e912e9498ed5442c097af5,Campaign Office,POINT (-70.61707 -33.39269)


In [20]:
pois_grid = gpd.sjoin(pois_gdf, amb[['geometry']], predicate='within', how='inner')
pois_in_grid = pois_grid.groupby('venue_id', group_keys=False).first().reset_index()
pois_in_grid

,venue_id,category,geometry,index_right
0,4af8ffeaf964a520cd1022e3,Other Great Outdoors,POINT (-70.66046 -33.44616),26
1,4b4463faf964a52078f425e3,Sushi Restaurant,POINT (-70.51793 -33.3738),4
2,4b44abe7f964a5201ef925e3,Sushi Restaurant,POINT (-70.57765 -33.41222),4
3,4b44ad89f964a52040f925e3,Office,POINT (-70.57276 -33.40448),4
4,4b44b566f964a5200efa25e3,Sandwich Place,POINT (-70.62263 -33.43098),31
...,...,...,...,...
92089,52e8241811d24e40d89978d0,Home (private),POINT (-70.60383 -33.47606),16
92090,52e8d7ad498e459461a0b33b,Bus Line,POINT (-70.72786 -33.42376),3
92091,52e90de1498ed1ed01b47aaf,Bus Line,POINT (-70.84306 -33.44869),2
92092,52e912e9498ed5442c097af5,Campaign Office,POINT (-70.61707 -33.39269),15


Import foursquare data (raw checkins)

In [21]:
check_ins = pd.read_csv('../data/other_format/foursquare/raw_Checkins_anonymized.txt', sep='\t', names=['user_id', 'venue_id', 'datetime', 'utc_offset']) # dd.read_csv is not working with the current version of pyarrow
check_ins

,user_id,venue_id,datetime,utc_offset
0,546830,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:06 +0000 2012,240
1,822121,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,180
2,2277773,4a85b1b3f964a520eefe1fe3,Tue Apr 03 18:00:08 +0000 2012,-240
3,208842,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300
4,1139878,4d9254ef62ad5481fa6e6a4b,Tue Apr 03 18:00:08 +0000 2012,-180
...,...,...,...,...
90048622,2528387,508835fee4b089a9f078d225,Wed Jan 29 16:44:23 +0000 2014,120
90048623,2678430,4f5638f1e4b09c08b95bf21c,Wed Jan 29 16:44:23 +0000 2014,120
90048624,2469868,4be139c2c1732d7f797c5b9a,Wed Jan 29 16:44:24 +0000 2014,120
90048625,1562566,4c25e56aa852c928e69be56c,Wed Jan 29 16:44:24 +0000 2014,120


Cross both dataframes to get one data frame with user_id, x, y, datetime

In [22]:
amb_check_ins = check_ins[check_ins.venue_id.isin(pois_in_grid.venue_id)]
amb_check_ins.shape

(1306426, 4)

In [28]:
amb_check_ins

,user_id,venue_id,datetime,utc_offset
56,44093,4dc0367c6a23e5a549e68d9b,Tue Apr 03 18:00:22 +0000 2012,-180
82,980044,4bb8ddd31261d13ab608e998,Tue Apr 03 18:00:27 +0000 2012,-180
83,640995,4f7200c9e4b0e30fde677e23,Tue Apr 03 18:00:28 +0000 2012,-180
179,969261,4d386aa93ffba1433b405c56,Tue Apr 03 18:00:55 +0000 2012,-180
180,1424666,4b6f2cf8f964a52013e22ce3,Tue Apr 03 18:00:55 +0000 2012,-180
...,...,...,...,...
90048214,290903,4f71bccee4b037626f904303,Wed Jan 29 16:41:35 +0000 2014,-180
90048333,21455,52d6ccb7498ed1693ba6f05d,Wed Jan 29 16:42:18 +0000 2014,-180
90048359,968028,4b65d266f964a5200b022be3,Wed Jan 29 16:42:29 +0000 2014,-180
90048399,2194463,4fea56c0e4b09f197e9379c5,Wed Jan 29 16:42:46 +0000 2014,-180


In [30]:
check_ins_points = amb_check_ins.merge(pois_in_grid, left_on='venue_id', right_on='venue_id', how='left') 
check_ins_points = check_ins_points.drop(columns=['utc_offset', 'category', 'index_right']) 
check_ins_points

,user_id,venue_id,datetime,geometry
0,44093,4dc0367c6a23e5a549e68d9b,Tue Apr 03 18:00:22 +0000 2012,POINT (-70.63263 -33.44886)
1,980044,4bb8ddd31261d13ab608e998,Tue Apr 03 18:00:27 +0000 2012,POINT (-70.64572 -33.44302)
2,640995,4f7200c9e4b0e30fde677e23,Tue Apr 03 18:00:28 +0000 2012,POINT (-70.60891 -33.42188)
3,969261,4d386aa93ffba1433b405c56,Tue Apr 03 18:00:55 +0000 2012,POINT (-70.64905 -33.4395)
4,1424666,4b6f2cf8f964a52013e22ce3,Tue Apr 03 18:00:55 +0000 2012,POINT (-70.65385 -33.4429)
...,...,...,...,...
1306421,290903,4f71bccee4b037626f904303,Wed Jan 29 16:41:35 +0000 2014,POINT (-70.5789 -33.61)
1306422,21455,52d6ccb7498ed1693ba6f05d,Wed Jan 29 16:42:18 +0000 2014,POINT (-70.61924 -33.42893)
1306423,968028,4b65d266f964a5200b022be3,Wed Jan 29 16:42:29 +0000 2014,POINT (-70.5573 -33.56931)
1306424,2194463,4fea56c0e4b09f197e9379c5,Wed Jan 29 16:42:46 +0000 2014,POINT (-70.60752 -33.41747)


Add h3 cell (from latitude and longitude) to each check in

In [32]:
import h3
res = 12
check_ins_points['h3_cell'] = check_ins_points['geometry'].apply(lambda point: h3.latlng_to_cell(point.y, point.x, res))
check_ins_points

,user_id,venue_id,datetime,geometry,h3_cell
0,44093,4dc0367c6a23e5a549e68d9b,Tue Apr 03 18:00:22 +0000 2012,POINT (-70.63263 -33.44886),8cb2c554c0747ff
1,980044,4bb8ddd31261d13ab608e998,Tue Apr 03 18:00:27 +0000 2012,POINT (-70.64572 -33.44302),8cb2c554136a1ff
2,640995,4f7200c9e4b0e30fde677e23,Tue Apr 03 18:00:28 +0000 2012,POINT (-70.60891 -33.42188),8cb2c55680cd3ff
3,969261,4d386aa93ffba1433b405c56,Tue Apr 03 18:00:55 +0000 2012,POINT (-70.64905 -33.4395),8cb2c5541a8e5ff
4,1424666,4b6f2cf8f964a52013e22ce3,Tue Apr 03 18:00:55 +0000 2012,POINT (-70.65385 -33.4429),8cb2c5541394bff
...,...,...,...,...,...
1306421,290903,4f71bccee4b037626f904303,Wed Jan 29 16:41:35 +0000 2014,POINT (-70.5789 -33.61),8cb2c5733871bff
1306422,21455,52d6ccb7498ed1693ba6f05d,Wed Jan 29 16:42:18 +0000 2014,POINT (-70.61924 -33.42893),8cb2c556aa4ddff
1306423,968028,4b65d266f964a5200b022be3,Wed Jan 29 16:42:29 +0000 2014,POINT (-70.5573 -33.56931),8cb2c509b0e93ff
1306424,2194463,4fea56c0e4b09f197e9379c5,Wed Jan 29 16:42:46 +0000 2014,POINT (-70.60752 -33.41747),8cb2c556815c5ff


Build the trips, sorting the table by users and date, where each trip is inferred as the transfer that a user made from the location of a check-in to the next location they visited (in chronological order).

In [53]:
check_ins_points.index.name = 'checkin_id'
checkins_sorted = check_ins_points.sort_values(by=['user_id', 'datetime'], ascending=[True, True])
checkins_sorted

,user_id,venue_id,datetime,geometry,h3_cell
checkin_id,,,,,
95547,230,4b549b4ef964a52060c227e3,Fri Apr 20 14:49:25 +0000 2012,POINT (-70.57852 -33.4022),8cb2c556db34dff
520250,230,4c2231a39a67a593ee0ddc87,Fri Aug 10 15:04:41 +0000 2012,POINT (-70.59798 -33.51787),8cb2c50939a2dff
776542,230,4b65adfef964a52047f92ae3,Fri Dec 21 12:13:55 +0000 2012,POINT (-70.67994 -33.45474),8cb2c5543754bff
776722,230,5072d1b5e4b0f322440bc73f,Fri Dec 21 12:59:17 +0000 2012,POINT (-70.64836 -33.42702),8cb2c55452c45ff
787339,230,4b59c564f964a5202c9728e3,Fri Dec 28 15:07:04 +0000 2012,POINT (-70.62704 -33.42828),8cb2c556aa861ff
...,...,...,...,...,...
1262638,2733324,4ba0f706f964a520ad8a37e3,Wed Nov 13 11:14:51 +0000 2013,POINT (-70.55311 -33.4336),8cb2c519b79c7ff
1263132,2733324,4b82bb3cf964a52007e130e3,Wed Nov 13 16:03:38 +0000 2013,POINT (-70.60599 -33.43222),8cb2c556b8559ff
1263226,2733324,4bba03851261d13af117ea98,Wed Nov 13 16:38:23 +0000 2013,POINT (-70.51575 -33.35767),8cb2c51800729ff


In [68]:
def shift(df):
    origin = df.rename({'geometry': 'o_point','h3_cell':'o_h3_cell','datetime':'o_time'}, axis=1)[['user_id', 'o_point', 'o_h3_cell', 'o_time']].shift()
    destination = df.rename({'user_id': 'user_id_d', 'geometry': 'd_point','h3_cell':'d_h3_cell','datetime':'d_time'}, axis=1)[['user_id_d', 'd_point', 'd_h3_cell', 'd_time']]
    trips = (origin.join(destination)
             .dropna()
             .pipe(lambda x: x[x.user_id == x.user_id_d])
            )
    return trips

In [69]:
user_trip_counts = shift(checkins_sorted).reset_index()
user_trip_counts.index.name="trip_id"

In [70]:
user_trip_counts

,checkin_id,user_id,o_point,o_h3_cell,o_time,user_id_d,d_point,d_h3_cell,d_time
trip_id,,,,,,,,,
0,520250,230.0,POINT (-70.57852 -33.4022),8cb2c556db34dff,Fri Apr 20 14:49:25 +0000 2012,230,POINT (-70.59798 -33.51787),8cb2c50939a2dff,Fri Aug 10 15:04:41 +0000 2012
1,776542,230.0,POINT (-70.59798 -33.51787),8cb2c50939a2dff,Fri Aug 10 15:04:41 +0000 2012,230,POINT (-70.67994 -33.45474),8cb2c5543754bff,Fri Dec 21 12:13:55 +0000 2012
2,776722,230.0,POINT (-70.67994 -33.45474),8cb2c5543754bff,Fri Dec 21 12:13:55 +0000 2012,230,POINT (-70.64836 -33.42702),8cb2c55452c45ff,Fri Dec 21 12:59:17 +0000 2012
3,787339,230.0,POINT (-70.64836 -33.42702),8cb2c55452c45ff,Fri Dec 21 12:59:17 +0000 2012,230,POINT (-70.62704 -33.42828),8cb2c556aa861ff,Fri Dec 28 15:07:04 +0000 2012
4,865173,230.0,POINT (-70.62704 -33.42828),8cb2c556aa861ff,Fri Dec 28 15:07:04 +0000 2012,230,POINT (-70.55216 -33.41351),8cb2c5198793dff,Fri Feb 01 01:02:08 +0000 2013
...,...,...,...,...,...,...,...,...,...
1265333,1262638,2733324.0,POINT (-70.5786 -33.60102),8cb2c57302347ff,Wed Jan 29 13:29:56 +0000 2014,2733324,POINT (-70.55311 -33.4336),8cb2c519b79c7ff,Wed Nov 13 11:14:51 +0000 2013
1265334,1263132,2733324.0,POINT (-70.55311 -33.4336),8cb2c519b79c7ff,Wed Nov 13 11:14:51 +0000 2013,2733324,POINT (-70.60599 -33.43222),8cb2c556b8559ff,Wed Nov 13 16:03:38 +0000 2013
1265335,1263226,2733324.0,POINT (-70.60599 -33.43222),8cb2c556b8559ff,Wed Nov 13 16:03:38 +0000 2013,2733324,POINT (-70.51575 -33.35767),8cb2c51800729ff,Wed Nov 13 16:38:23 +0000 2013


In [71]:
trips = user_trip_counts.drop("checkin_id", axis=1)
trips

,user_id,o_point,o_h3_cell,o_time,user_id_d,d_point,d_h3_cell,d_time
trip_id,,,,,,,,
0,230.0,POINT (-70.57852 -33.4022),8cb2c556db34dff,Fri Apr 20 14:49:25 +0000 2012,230,POINT (-70.59798 -33.51787),8cb2c50939a2dff,Fri Aug 10 15:04:41 +0000 2012
1,230.0,POINT (-70.59798 -33.51787),8cb2c50939a2dff,Fri Aug 10 15:04:41 +0000 2012,230,POINT (-70.67994 -33.45474),8cb2c5543754bff,Fri Dec 21 12:13:55 +0000 2012
2,230.0,POINT (-70.67994 -33.45474),8cb2c5543754bff,Fri Dec 21 12:13:55 +0000 2012,230,POINT (-70.64836 -33.42702),8cb2c55452c45ff,Fri Dec 21 12:59:17 +0000 2012
3,230.0,POINT (-70.64836 -33.42702),8cb2c55452c45ff,Fri Dec 21 12:59:17 +0000 2012,230,POINT (-70.62704 -33.42828),8cb2c556aa861ff,Fri Dec 28 15:07:04 +0000 2012
4,230.0,POINT (-70.62704 -33.42828),8cb2c556aa861ff,Fri Dec 28 15:07:04 +0000 2012,230,POINT (-70.55216 -33.41351),8cb2c5198793dff,Fri Feb 01 01:02:08 +0000 2013
...,...,...,...,...,...,...,...,...
1265333,2733324.0,POINT (-70.5786 -33.60102),8cb2c57302347ff,Wed Jan 29 13:29:56 +0000 2014,2733324,POINT (-70.55311 -33.4336),8cb2c519b79c7ff,Wed Nov 13 11:14:51 +0000 2013
1265334,2733324.0,POINT (-70.55311 -33.4336),8cb2c519b79c7ff,Wed Nov 13 11:14:51 +0000 2013,2733324,POINT (-70.60599 -33.43222),8cb2c556b8559ff,Wed Nov 13 16:03:38 +0000 2013
1265335,2733324.0,POINT (-70.60599 -33.43222),8cb2c556b8559ff,Wed Nov 13 16:03:38 +0000 2013,2733324,POINT (-70.51575 -33.35767),8cb2c51800729ff,Wed Nov 13 16:38:23 +0000 2013


In [73]:
trips['o_lon'] = trips['o_point'].apply(lambda point: point.x)
trips['o_lat'] = trips['o_point'].apply(lambda point: point.y)
trips['d_lon'] = trips['d_point'].apply(lambda point: point.x)
trips['d_lat'] = trips['d_point'].apply(lambda point: point.y)
trips

,user_id,o_point,o_h3_cell,o_time,user_id_d,d_point,d_h3_cell,d_time,o_lon,o_lat,d_lon,d_lat
trip_id,,,,,,,,,,,,
0,230.0,POINT (-70.57852 -33.4022),8cb2c556db34dff,Fri Apr 20 14:49:25 +0000 2012,230,POINT (-70.59798 -33.51787),8cb2c50939a2dff,Fri Aug 10 15:04:41 +0000 2012,-70.578518,-33.402201,-70.597980,-33.517874
1,230.0,POINT (-70.59798 -33.51787),8cb2c50939a2dff,Fri Aug 10 15:04:41 +0000 2012,230,POINT (-70.67994 -33.45474),8cb2c5543754bff,Fri Dec 21 12:13:55 +0000 2012,-70.597980,-33.517874,-70.679939,-33.454735
2,230.0,POINT (-70.67994 -33.45474),8cb2c5543754bff,Fri Dec 21 12:13:55 +0000 2012,230,POINT (-70.64836 -33.42702),8cb2c55452c45ff,Fri Dec 21 12:59:17 +0000 2012,-70.679939,-33.454735,-70.648357,-33.427018
3,230.0,POINT (-70.64836 -33.42702),8cb2c55452c45ff,Fri Dec 21 12:59:17 +0000 2012,230,POINT (-70.62704 -33.42828),8cb2c556aa861ff,Fri Dec 28 15:07:04 +0000 2012,-70.648357,-33.427018,-70.627044,-33.428281
4,230.0,POINT (-70.62704 -33.42828),8cb2c556aa861ff,Fri Dec 28 15:07:04 +0000 2012,230,POINT (-70.55216 -33.41351),8cb2c5198793dff,Fri Feb 01 01:02:08 +0000 2013,-70.627044,-33.428281,-70.552159,-33.413510
...,...,...,...,...,...,...,...,...,...,...,...,...
1265333,2733324.0,POINT (-70.5786 -33.60102),8cb2c57302347ff,Wed Jan 29 13:29:56 +0000 2014,2733324,POINT (-70.55311 -33.4336),8cb2c519b79c7ff,Wed Nov 13 11:14:51 +0000 2013,-70.578600,-33.601019,-70.553106,-33.433601
1265334,2733324.0,POINT (-70.55311 -33.4336),8cb2c519b79c7ff,Wed Nov 13 11:14:51 +0000 2013,2733324,POINT (-70.60599 -33.43222),8cb2c556b8559ff,Wed Nov 13 16:03:38 +0000 2013,-70.553106,-33.433601,-70.605987,-33.432220
1265335,2733324.0,POINT (-70.60599 -33.43222),8cb2c556b8559ff,Wed Nov 13 16:03:38 +0000 2013,2733324,POINT (-70.51575 -33.35767),8cb2c51800729ff,Wed Nov 13 16:38:23 +0000 2013,-70.605987,-33.432220,-70.515747,-33.357667


In [74]:
trips_file = trips.drop(["o_point", "d_point"], axis=1)
trips_file

,user_id,o_h3_cell,o_time,user_id_d,d_h3_cell,d_time,o_lon,o_lat,d_lon,d_lat
trip_id,,,,,,,,,,
0,230.0,8cb2c556db34dff,Fri Apr 20 14:49:25 +0000 2012,230,8cb2c50939a2dff,Fri Aug 10 15:04:41 +0000 2012,-70.578518,-33.402201,-70.597980,-33.517874
1,230.0,8cb2c50939a2dff,Fri Aug 10 15:04:41 +0000 2012,230,8cb2c5543754bff,Fri Dec 21 12:13:55 +0000 2012,-70.597980,-33.517874,-70.679939,-33.454735
2,230.0,8cb2c5543754bff,Fri Dec 21 12:13:55 +0000 2012,230,8cb2c55452c45ff,Fri Dec 21 12:59:17 +0000 2012,-70.679939,-33.454735,-70.648357,-33.427018
3,230.0,8cb2c55452c45ff,Fri Dec 21 12:59:17 +0000 2012,230,8cb2c556aa861ff,Fri Dec 28 15:07:04 +0000 2012,-70.648357,-33.427018,-70.627044,-33.428281
4,230.0,8cb2c556aa861ff,Fri Dec 28 15:07:04 +0000 2012,230,8cb2c5198793dff,Fri Feb 01 01:02:08 +0000 2013,-70.627044,-33.428281,-70.552159,-33.413510
...,...,...,...,...,...,...,...,...,...,...
1265333,2733324.0,8cb2c57302347ff,Wed Jan 29 13:29:56 +0000 2014,2733324,8cb2c519b79c7ff,Wed Nov 13 11:14:51 +0000 2013,-70.578600,-33.601019,-70.553106,-33.433601
1265334,2733324.0,8cb2c519b79c7ff,Wed Nov 13 11:14:51 +0000 2013,2733324,8cb2c556b8559ff,Wed Nov 13 16:03:38 +0000 2013,-70.553106,-33.433601,-70.605987,-33.432220
1265335,2733324.0,8cb2c556b8559ff,Wed Nov 13 16:03:38 +0000 2013,2733324,8cb2c51800729ff,Wed Nov 13 16:38:23 +0000 2013,-70.605987,-33.432220,-70.515747,-33.357667


In [76]:
trips_to_file = trips_file[['user_id', 'o_lon', 'o_lat', 'd_lon', 'd_lat', 'o_h3_cell', 'd_h3_cell', 'o_time', 'd_time']]
trips_to_file

,user_id,o_lon,o_lat,d_lon,d_lat,o_h3_cell,d_h3_cell,o_time,d_time
trip_id,,,,,,,,,
0,230.0,-70.578518,-33.402201,-70.597980,-33.517874,8cb2c556db34dff,8cb2c50939a2dff,Fri Apr 20 14:49:25 +0000 2012,Fri Aug 10 15:04:41 +0000 2012
1,230.0,-70.597980,-33.517874,-70.679939,-33.454735,8cb2c50939a2dff,8cb2c5543754bff,Fri Aug 10 15:04:41 +0000 2012,Fri Dec 21 12:13:55 +0000 2012
2,230.0,-70.679939,-33.454735,-70.648357,-33.427018,8cb2c5543754bff,8cb2c55452c45ff,Fri Dec 21 12:13:55 +0000 2012,Fri Dec 21 12:59:17 +0000 2012
3,230.0,-70.648357,-33.427018,-70.627044,-33.428281,8cb2c55452c45ff,8cb2c556aa861ff,Fri Dec 21 12:59:17 +0000 2012,Fri Dec 28 15:07:04 +0000 2012
4,230.0,-70.627044,-33.428281,-70.552159,-33.413510,8cb2c556aa861ff,8cb2c5198793dff,Fri Dec 28 15:07:04 +0000 2012,Fri Feb 01 01:02:08 +0000 2013
...,...,...,...,...,...,...,...,...,...
1265333,2733324.0,-70.578600,-33.601019,-70.553106,-33.433601,8cb2c57302347ff,8cb2c519b79c7ff,Wed Jan 29 13:29:56 +0000 2014,Wed Nov 13 11:14:51 +0000 2013
1265334,2733324.0,-70.553106,-33.433601,-70.605987,-33.432220,8cb2c519b79c7ff,8cb2c556b8559ff,Wed Nov 13 11:14:51 +0000 2013,Wed Nov 13 16:03:38 +0000 2013
1265335,2733324.0,-70.605987,-33.432220,-70.515747,-33.357667,8cb2c556b8559ff,8cb2c51800729ff,Wed Nov 13 16:03:38 +0000 2013,Wed Nov 13 16:38:23 +0000 2013


Save as a parquet file

In [78]:
trips_to_file.to_parquet('../data/unified_format/santiago_foursquare_unitrip.parquet', engine='pyarrow', compression='snappy')

## Code to aggregate trips and create flows in unitrip format (temp in notebook)

#### Aggregate trips using h3 cells, with a specific level of resolution and filtering with a minimum number of trips per flow

Load the parquet to a dataframe with the important series

In [84]:
santiago_unitrip = pd.read_parquet('../data/unified_format/santiago_foursquare_unitrip.parquet', columns=['user_id', 'o_h3_cell', 'd_h3_cell'])
santiago_unitrip

,user_id,o_h3_cell,d_h3_cell
trip_id,,,
0,230.0,8cb2c556db34dff,8cb2c50939a2dff
1,230.0,8cb2c50939a2dff,8cb2c5543754bff
2,230.0,8cb2c5543754bff,8cb2c55452c45ff
3,230.0,8cb2c55452c45ff,8cb2c556aa861ff
4,230.0,8cb2c556aa861ff,8cb2c5198793dff
...,...,...,...
1265333,2733324.0,8cb2c57302347ff,8cb2c519b79c7ff
1265334,2733324.0,8cb2c519b79c7ff,8cb2c556b8559ff
1265335,2733324.0,8cb2c556b8559ff,8cb2c51800729ff


Set the h3 cell columns to a parent resolution

In [86]:
flow_res = 7
santiago_unitrip['o_h3_cell'] = santiago_unitrip['o_h3_cell'].apply(lambda cell: h3.cell_to_parent(cell, flow_res))
santiago_unitrip['d_h3_cell'] = santiago_unitrip['d_h3_cell'].apply(lambda cell: h3.cell_to_parent(cell, flow_res))
santiago_unitrip

,user_id,o_h3_cell,d_h3_cell
trip_id,,,
0,230.0,87b2c556dffffff,87b2c5093ffffff
1,230.0,87b2c5093ffffff,87b2c5543ffffff
2,230.0,87b2c5543ffffff,87b2c5545ffffff
3,230.0,87b2c5545ffffff,87b2c556affffff
4,230.0,87b2c556affffff,87b2c5198ffffff
...,...,...,...
1265333,2733324.0,87b2c5730ffffff,87b2c519bffffff
1265334,2733324.0,87b2c519bffffff,87b2c556bffffff
1265335,2733324.0,87b2c556bffffff,87b2c5180ffffff


Trips from one cell to the same cell are eliminated, and the same OD trips made by a user are grouped together.

In [94]:
filtered_unitrip = (santiago_unitrip
                    .pipe(lambda x: x[x.o_h3_cell != x.d_h3_cell])
                    .groupby(['user_id', 'o_h3_cell', 'd_h3_cell'])
                    .size()
                    )
filtered_unitrip.name = 'n_trips'
filtered_unitrip = filtered_unitrip.reset_index()
filtered_unitrip.index.name="trip_id"
filtered_unitrip

,user_id,o_h3_cell,d_h3_cell,n_trips
trip_id,,,,
0,230.0,87b2c5093ffffff,87b2c5543ffffff,1
1,230.0,87b2c5198ffffff,87b2c5464ffffff,1
2,230.0,87b2c5198ffffff,87b2c5543ffffff,1
3,230.0,87b2c540cffffff,87b2c5543ffffff,1
4,230.0,87b2c542dffffff,87b2c5541ffffff,1
...,...,...,...,...
550102,2733324.0,87b2c5730ffffff,87b2c5464ffffff,1
550103,2733324.0,87b2c5730ffffff,87b2c5541ffffff,1
550104,2733324.0,87b2c5730ffffff,87b2c556affffff,1


Aggregate trips of the same OD and with a minimun number of trips

In [95]:
minimun_trips = 8
flows = (filtered_unitrip
         [filtered_unitrip.o_h3_cell != filtered_unitrip.d_h3_cell]
         .groupby(['o_h3_cell', 'd_h3_cell'])
         ['n_trips']
         .sum()
         .reset_index()
         .pipe(lambda x: x[x.n_trips > minimun_trips])
         .rename({'n_trips': 'count'}, axis=1)
        )
flows

,o_h3_cell,d_h3_cell,count
53,87b2c5080ffffff,87b2c5081ffffff,9
58,87b2c5080ffffff,87b2c5086ffffff,13
109,87b2c5080ffffff,87b2c554cffffff,13
111,87b2c5080ffffff,87b2c554effffff,18
114,87b2c5080ffffff,87b2c5568ffffff,15
...,...,...,...
27431,87b2c5cdeffffff,87b2c5575ffffff,10
27449,87b2c5cdeffffff,87b2c5cd3ffffff,14
27450,87b2c5cdeffffff,87b2c5cd8ffffff,11
27452,87b2c5cdeffffff,87b2c5cdaffffff,16


Save as a parquet file

In [96]:
flows.to_parquet('../data/unified_format/santiago_foursquare_uniflow.parquet', engine='pyarrow', compression='snappy')

## Code to convert trips and flows (in unitrip format) to flowmap input (temp in notebook)

### Create the flowmap input from trips in unitrip format

Agreggate the trips to create flows

Create locations magnitude (like in santiago_foursquare) with: id(int), x and y coordinates from h3 cell centroid

Create locations flows (like in santiago_foursquare) with: id(int), with: origin and destination pointing locations coordinates, and the count of trips

### Create the flowmap input from flows in unitrip format (uniflow)

Create locations magnitude (like in santiago_foursquare) with: id(int), x and y coordinates from h3 cell centroid

Create locations flows (like in santiago_foursquare) with: id(int), with: origin and destination pointing locations coordinates, and the count of trips

## Visualizing Foursquare data in Flowmap using the Unitrip format

Load the raw data of foursquare

Run the methods of the module to:
- Convert to unitrip format
- Unitrio to uniflow
- Uniflow to flowmap input

Save the dataframes as flowmap input

Present the visualizaton url